# MIIII

In [ ]:
import jax, optax, yaml
import jax.numpy as jnp
from jax import jit, random, tree_util, vmap, grad, value_and_grad
from jax.lib import xla_client
from functional import partial
from tqdm import tqdm
import tikz
from typing import Any, Tuple, List, Dict, Iterator
from oeis import A000032


from src import args_fn, apply_fn, init_fn
import esch

In [ ]:
import oeis as OEIS

In [ ]:
def cross_entropy_fn(params, x, y):  # todo: weight by prime frquency
    y_pred = apply_fn(params, x)
    loss = -jnp.mean(y * jnp.log(y_pred + 1e-8))
    return loss

In [ ]:
@jit
def update_fn(params, state, x, y):
    loss, grads = jax.value_and_grad(loss_fn)(params, x, y)
    updates, state = opt.update(grads, state, params)
    params = optax.apply_updates(params, updates)
    return params, state, loss

In [ ]:
pbar = tqdm(range(conf["epochs"]))
losses = jnp.zeros(conf["epochs"])
for epoch in pbar:
    params, state, loss = update_fn(params, state, x, y)
    losses = losses.at[epoch].add(loss)

In [ ]:
info = dict(title="Training curves", xlab="Epoch", ylab="MSE")
fig = esch.curves_fn([losses, losses - 0.01, losses + 0.01, losses + 0.02], info)
fig.show()

In [ ]:
y.shape

In [ ]:
def spiral_fn(v):
    # floor sqrt of length of v
    n = jnp.sqrt(v.shape[0]).astype(jnp.int32)
    v, lst = v[: n * n].reshape(n, n), [v[:n]]
    print(lst)
    while v.size:
        v = jnp.rot90(v[1:], 1)
        lst.append(v[0])
    return jnp.concatenate(lst).reshape((n, n))

In [ ]:
spiral_fn(jnp.arange(100))